## Profilers for Python

In this session we will cover two kinds of profiling: deterministic and statistical profiling. 

We'll use both simpler approaches to profile software and more involved softwares.

### Built-in Jupyter methods

In [ ]:
import pandas as pd
import numpy as np


df = pd.DataFrame({
        "a": np.random.randn(1000),
        "b": np.random.randn(1000),
        "N": np.random.randint(100, 1000, (1000)),
        "x": "x",
    })
df

In [ ]:
def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N

    for i in range(N):
        s += f(a + i * dx)

    return s * dx

In [ ]:
%timeit df.apply(lambda x: integrate_f(x["a"], x["b"], x["N"]), axis=1)

In [ ]:
%prun -l 4 df.apply(lambda x: integrate_f(x["a"], x["b"], x["N"]), axis=1)  # noqa E999

### Tracing

`sys.setprofile` sets a trace function that's triggered every time the VM enters or exits both Python and C functions.

In [ ]:
# ⚠️ Run this code on your Python CLI

# def fib(n):
#     i, f1, f2 = 1, 1, 1
#     while i < n:
#         f1, f2 = f2, f1 + f2
#         i += 1
#     return f1

# import opcode


# def show_trace(frame, event, arg):
#     frame.f_trace_opcodes = True
#     code = frame.f_code
#     offset = frame.f_lasti

#     print(f"| {event:10} | {str(arg):>4} |", end=' ')
#     print(f"{frame.f_lineno:>4} | {frame.f_lasti:>6} |", end=' ')
#     print(f"{opcode.opname[code.co_code[offset]]:<18} | {str(frame.f_locals):<35} |")
#     return show_trace

# import sys

# header = f"| {'event':10} | {'arg':>4} | line | offset | {'opcode':^18} | {'locals':^35} |"
# print(header)
# sys.settrace(show_trace)
# fib(3)
# sys.settrace(None)

### Deterministic profiling

#### cProfile

- Offers you both the total running time of cProfile.run(statement, filename=None, sort=-1)a software as well as specific function calls and times
- Nice integration with GUI tools and pstats

In [ ]:
import cProfile

# cProfile.run(statement, filename=None, sort=-1)

You can pass python code or a function name that you want to profile as a string to the statement argument.

In [ ]:
import numpy as np

cProfile.run("2**200000")

- ncalls : Shows the number of calls made
- tottime: Total time taken by the given function. Note that the time made in calls to sub-functions are excluded.
- percall: Total time / No of calls. ( remainder is left out )
- cumtime: Unlike tottime, this includes time spent in this and all subfunctions that the higher-level function calls. It is most useful and is accurate for recursive functions.
- The percall following cumtime is calculated as the quotient of cumtime divided by primitive calls. The primitive calls include all the calls that were not included through recursion.

In [ ]:
def add_emoji():
    arr=[]
    arr.append('🔥')

def multiply():
    arr=[]
    for i in range(0,400000):
        arr.append(i * 2)
        add_emoji()

def main():
    multiply()
    print('end')

if __name__ == '__main__':
    cProfile.run('main()')

You can save the data using the following:

In [ ]:
import cProfile, pstats
profiler = cProfile.Profile()
stats = pstats.Stats(profiler)
stats.dump_stats('/content/export-data')

In [ ]:
And use a GUI to visualize it called snakeviz:

In [ ]:
# installing the module
!pip install snakeviz

In [ ]:
# load it on the notebook
%load_ext snakeviz <filename>

In [ ]:
# opens snakeviz
%snakeviz main()

cProfile has a lot more to offer and I recommend checking the [Python docs](https://docs.python.org/3/library/profile.html#module-cProfile) to learn more about its specific functions.

### Statistical profiling

Samples the program counter at regular intervals. The numbers will be statistical approximations instead of exact numbers because of the several process in place.

- Less data to analyze
- Smaller profiling footprint

#### pprofile

pprofile offers both deterministic and statistical modes for profiling. We're going to take a look in the statistical mode:

In your CLI, after installing `pprofile` run it in the statistical mode:

``

In [ ]:
# ⚠️ Run this code on your Python CLI

# import threading
# import time


# def func():
#     time.sleep(1)

# def func2():
#     pass

# t1 = threading.Thread(target=func)
# t2 = threading.Thread(target=func)
# t1.start()
# t2.start()
# (func(), func2())
# t1.join()
# t2.join()